In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [1]:
!nvidia-smi

Tue May 11 23:01:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 13.95 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 8.8MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 7.5MB 50.0MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp37-none-any.whl size=54235 sha256=52f804e33f04d534312485e4b64bd744ca4259177615709f82a314c85986daea
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.20.71 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
['tests', '.git', 'demo.ipynb', 'glue_examples', 'demo_tuning_hyperparams.ipynb', 'other_examples', 'README.md', 'setup.py', 'LICENSE', 'be

In [3]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.8.1+cu101
GPU: Tesla T4


In [56]:
from google.colab import files
uploaded = files.upload()

Saving EI-reg-En-anger-train.txt to EI-reg-En-anger-train (1).txt
Saving EI-reg-En-fear-train.txt to EI-reg-En-fear-train (1).txt
Saving EI-reg-En-joy-train.txt to EI-reg-En-joy-train (1).txt
Saving EI-reg-En-sadness-train.txt to EI-reg-En-sadness-train (1).txt


In [5]:
import io

In [71]:
train_anger = pd.read_csv('EI-reg-En-anger-train.txt', sep='\t', encoding='utf8', keep_default_na=False)
train_fear = pd.read_csv('EI-reg-En-fear-train.txt', sep='\t', encoding='utf8', keep_default_na=False)
train_joy = pd.read_csv('EI-reg-En-joy-train.txt', sep='\t', encoding='utf8', keep_default_na=False)
train_sadness = pd.read_csv('EI-reg-En-sadness-train.txt', sep='\t', encoding='utf8', keep_default_na=False)



In [74]:
training = pd.concat([train_anger, train_fear, train_joy, train_sadness], ignore_index=True)

In [76]:
def text_cleaning(input_text):
    input_text = re.sub(u'http\S+|@\S+', ' ', input_text)
    input_text = re.sub(u'^(.{140}).*$', '\g<1>', input_text)
    #input_text = contractions.fix(input_text)
    #input_text = unidecode.unidecode(input_text)
    input_text = re.sub(u'[^a-zA-Z]', ' ', input_text)
    #input_text = stop
    input_text = input_text.lower()
    output_text = ' '.join(input_text.split())
    return output_text

In [81]:
import re
from sklearn.model_selection import train_test_split

In [79]:
training.insert(
    loc=1,
    column='cleaned_Tweet',
    value= training['Tweet'].map(text_cleaning)
)

In [82]:
y=training['Affect Dimension'].values
X=training['cleaned_Tweet'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4971,) (4971,) (2131,) (2131,)


In [83]:
# check the category distribution in the train and test set

import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)))

[['anger' 'fear' 'joy' 'sadness']
 [1199 1567 1124 1081]]
[['anger' 'fear' 'joy' 'sadness']
 [502 685 492 452]]


In [84]:
from bert_sklearn import BertClassifier
model = BertClassifier()    
print(model)
model.fit(X_train, y_train)

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)


100%|██████████| 231508/231508 [00:00<00:00, 892635.30B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 131804.46B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 4474, validation data size: 497



...

/content/bert-sklearn/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  next_m.mul_(beta1).add_(1 - beta1, grad)




Epoch 1, Train loss: 0.7863, Val loss: 0.4363, Val accy: 85.11%





Epoch 2, Train loss: 0.2815, Val loss: 0.3907, Val accy: 85.51%





Epoch 3, Train loss: 0.1602, Val loss: 0.4302, Val accy: 85.71%



BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=True, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None,
               label_list=array(['anger', 'fear', 'joy', 'sadness'], dtype=object),
               learning_rate=2e-05, local_rank=-1, logfile='bert_sklearn.log',
               loss_scale=0, max_seq_length=128, num_mlp_hiddens=500,
               num_mlp_layers=0, random_state=42, restore_file=None,
               train_batch_size=32, use_cuda=True, validation_fraction=0.1,
               warmup_proportion=0.1)

In [85]:
model.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))




Loss: 0.3525, Accuracy: 86.72%


86.71984983575786

In [86]:
y_pred = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [89]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm=confusion_matrix(y_test, y_pred, labels=['anger', 'fear', 'joy', 'sadness'])
print(cm)
print()

from sklearn.metrics import classification_report
target_names = ['anger', 'fear', 'joy', 'sadness']
print(classification_report(y_test, y_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



[[440  33   7  22]
 [ 33 592  10  50]
 [ 10  27 443  12]
 [ 29  45   5 373]]

              precision    recall  f1-score   support

       anger       0.86      0.88      0.87       502
        fear       0.85      0.86      0.86       685
         joy       0.95      0.90      0.93       492
     sadness       0.82      0.83      0.82       452

    accuracy                           0.87      2131
   macro avg       0.87      0.87      0.87      2131
weighted avg       0.87      0.87      0.87      2131



In [91]:
# BERT error analysis
err_cnt = 0
for i in range(0, len(y_test)):
  if (y_test[i]=="sadness" and y_pred[i]== "joy"):
    print(X_test[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

be joyful in hope patient in affliction faithful in prayer romans
don t be disheartened if you didn t get the cards you wanted it s not the end of the world e eh y you want me to cheer you up
y all tune into snapchat for beans funereal
even when we re fighting i m laughing i probably have serious ingrained issues
conflicting reports on mama ritamarley health mannn and i woke up today feeling like mi ole self rejoice now dis sadness
errors: 5


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,X_train,y_train,cv=5)
print(sum(scores)/len(scores))

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 3579, validation data size: 397



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))




Epoch 1, Train loss: 0.8492, Val loss: 0.3874, Val accy: 85.89%





Epoch 2, Train loss: 0.2893, Val loss: 0.3224, Val accy: 86.65%





Epoch 3, Train loss: 0.1559, Val loss: 0.3418, Val accy: 87.41%





Loss: 0.3854, Accuracy: 88.24%
Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 3580, validation data size: 397





Epoch 1, Train loss: 0.8382, Val loss: 0.3655, Val accy: 85.89%





Epoch 2, Train loss: 0.2954, Val loss: 0.3245, Val accy: 86.65%





Epoch 3, Train loss: 0.1706, Val loss: 0.3572, Val accy: 87.41%





Loss: 0.3590, Accuracy: 88.53%
Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 3580, validation data size: 397

